In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, numpy as np
from sklearn.model_selection import train_test_split

def normalize_data(X):
    """
    Normalize EEG per sample per channel.
    X: (N, C=14, T) -> normalized along T per (sample, channel)
    """
    mean = np.mean(X, axis=2, keepdims=True)
    std  = np.std(X, axis=2, keepdims=True)
    std[std == 0] = 1
    return (X - mean) / std

DATA_DIR = "/content/drive/MyDrive"  # adjust if needed
X = np.load(os.path.join(DATA_DIR, "DREAMERV_X.npy"), allow_pickle=True)
y = np.load(os.path.join(DATA_DIR, "DREAMERV_y.npy"), allow_pickle=True)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_test = normalize_data(X_test)

print(f"[INFO] X_test: {X_test.shape}, y_test: {y_test.shape} (normalized)")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[INFO] X_test: (34050, 14, 256), y_test: (34050,) (normalized)


In [ ]:
import os, time, numpy as np, pandas as pd
import torch, torch.nn as nn, torch.nn.functional as F
import torch.nn.utils.prune as prune
import torch.quantization as tq
from sklearn.metrics import accuracy_score, f1_score

DEVICE    = "cpu"
CKPT_CNN  = "/content/drive/MyDrive/cnn_model.pth"
CKPT_LSTM = "/content/drive/MyDrive/lstm_model.pth"


In [ ]:
class EEG_CNN(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.conv1 = nn.Conv1d(14, 32, kernel_size=3)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3)
        self.pool  = nn.AdaptiveAvgPool1d(1)
        self.fc1   = nn.Linear(64, 32)
        self.fc2   = nn.Linear(32, num_classes)
    def forward(self, x):  # [B,14,T]
        x = self.conv1(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x).squeeze(-1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

class EEG_LSTM(nn.Module):
    def __init__(self, input_size=14, hidden_size=64, num_layers=2, num_classes=2):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers  = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1  = nn.Linear(hidden_size, 32)
        self.fc2  = nn.Linear(32, num_classes)
    def forward(self, x):  # [B,T,14]
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size, device=x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = F.relu(self.fc1(out[:, -1, :]))
        return self.fc2(out)


In [ ]:
def load_model(name:str, ckpt:str):
    m = EEG_CNN().to(DEVICE) if name=="CNN" else EEG_LSTM().to(DEVICE)
    m.load_state_dict(torch.load(ckpt, map_location=DEVICE))
    m.eval()
    return m

def quantize_cpu(model: nn.Module):
    # dynamic quantization on Linear layers — CPU only
    return tq.quantize_dynamic(model.cpu(), {nn.Linear}, dtype=torch.qint8)

def prune_fc1(model_ctor, ckpt:str, amount:float=0.40):
    m = model_ctor().to(DEVICE)
    m.load_state_dict(torch.load(ckpt, map_location=DEVICE))
    prune.l1_unstructured(m.fc1, name="weight", amount=amount)
    m.eval()
    return m

def latency_ms(model, sample):
    model.eval()
    with torch.no_grad():
        t0 = time.time(); _ = model(sample); t1 = time.time()
    return (t1 - t0) * 1000.0

def eval_acc_f1(model_name:str, model, X_test=None, y_test=None, n_samples=200):
    """
    Assumes X_test is already normalized (Cell 0).
    If X_test/y_test are not available in this runtime, returns NaN.
    """
    if X_test is None or y_test is None:
        return float("nan"), float("nan")
    n = min(n_samples, len(X_test))
    Xb = torch.tensor(X_test[:n], dtype=torch.float32)
    yb = torch.tensor(y_test[:n], dtype=torch.long)
    if model_name == "CNN":
        logits = model(Xb)  # Xb: [N,14,T]
    else:
        logits = model(Xb.permute(0,2,1))  # LSTM expects [N,T,14]
    preds = torch.argmax(logits, dim=1).cpu().numpy()
    yt = yb.cpu().numpy()
    return float(accuracy_score(yt, preds)), float(f1_score(yt, preds, average="macro"))

def size_mb(path):
    return os.path.getsize(path)/1e6


In [ ]:
# Baseline
cnn_base   = load_model("CNN", CKPT_CNN)
sample_cnn = torch.randn(1, 14, 256)  # synthetic for latency
lat_b   = latency_ms(cnn_base, sample_cnn)
acc_b, f1_b = eval_acc_f1("CNN", cnn_base, globals().get("X_test"), globals().get("y_test"))

# Quantized (CPU)
cnn_quant = quantize_cpu(cnn_base)
torch.save(cnn_quant.state_dict(), "/content/drive/MyDrive/cnn_model_quantized.pth")
lat_q   = latency_ms(cnn_quant, sample_cnn)
acc_q, f1_q = eval_acc_f1("CNN", cnn_quant, globals().get("X_test"), globals().get("y_test"))

# Pruned (40% of fc1 weights)
cnn_pruned = prune_fc1(EEG_CNN, CKPT_CNN, amount=0.40)
torch.save(cnn_pruned.state_dict(), "/content/drive/MyDrive/cnn_model_pruned.pth")
lat_p   = latency_ms(cnn_pruned, sample_cnn)
acc_p, f1_p = eval_acc_f1("CNN", cnn_pruned, globals().get("X_test"), globals().get("y_test"))

df_cnn = pd.DataFrame([
    {"Model":"CNN","Variant":"Baseline","Latency_ms":lat_b,"Size_MB":size_mb(CKPT_CNN),"Accuracy":acc_b,"F1":f1_b},
    {"Model":"CNN","Variant":"Quantized","Latency_ms":lat_q,"Size_MB":size_mb("/content/drive/MyDrive/cnn_model_quantized.pth"),"Accuracy":acc_q,"F1":f1_q},
    {"Model":"CNN","Variant":"Pruned(0.4)","Latency_ms":lat_p,"Size_MB":size_mb("/content/drive/MyDrive/cnn_model_pruned.pth"),"Accuracy":acc_p,"F1":f1_p},
])
df_cnn.to_csv("/content/drive/MyDrive/week9_results_cnn.csv", index=False)
print("[INFO] saved /content/drive/MyDrive/week9_results_cnn.csv")
df_cnn


/tmp/ipython-input-116966612.py:9: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  return tq.quantize_dynamic(model.cpu(), {nn.Linear}, dtype=torch.qint8)


[INFO] saved /content/drive/MyDrive/week9_results_cnn.csv


,Model,Variant,Latency_ms,Size_MB,Accuracy,F1
0,CNN,Baseline,15.522480,0.042601,0.835,0.831937
1,CNN,Quantized,1.754761,0.037719,0.840,0.836801
2,CNN,Pruned(0.4),0.875950,0.051088,0.810,0.801234


In [ ]:
# Baseline
lstm_base   = load_model("LSTM", CKPT_LSTM)
sample_lstm = torch.randn(1, 256, 14)  # [B,T,14]
lat_b   = latency_ms(lstm_base, sample_lstm)
acc_b, f1_b = eval_acc_f1("LSTM", lstm_base, globals().get("X_test"), globals().get("y_test"))

# Quantized (CPU)
lstm_quant = quantize_cpu(lstm_base)
torch.save(lstm_quant.state_dict(), "/content/drive/MyDrive/lstm_model_quantized.pth")
lat_q   = latency_ms(lstm_quant, sample_lstm)
acc_q, f1_q = eval_acc_f1("LSTM", lstm_quant, globals().get("X_test"), globals().get("y_test"))

# Pruned (40% of fc1 weights)
lstm_pruned = prune_fc1(lambda: EEG_LSTM(), CKPT_LSTM, amount=0.40)
torch.save(lstm_pruned.state_dict(), "/content/drive/MyDrive/lstm_model_pruned.pth")
lat_p   = latency_ms(lstm_pruned, sample_lstm)
acc_p, f1_p = eval_acc_f1("LSTM", lstm_pruned, globals().get("X_test"), globals().get("y_test"))

df_lstm = pd.DataFrame([
    {"Model":"LSTM","Variant":"Baseline","Latency_ms":lat_b,"Size_MB":size_mb(CKPT_LSTM),"Accuracy":acc_b,"F1":f1_b},
    {"Model":"LSTM","Variant":"Quantized","Latency_ms":lat_q,"Size_MB":size_mb("/content/drive/MyDrive/lstm_model_quantized.pth"),"Accuracy":acc_q,"F1":f1_q},
    {"Model":"LSTM","Variant":"Pruned(0.4)","Latency_ms":lat_p,"Size_MB":size_mb("/content/drive/MyDrive/lstm_model_pruned.pth"),"Accuracy":acc_p,"F1":f1_p},
])
df_lstm.to_csv("/content/drive/MyDrive/week9_results_lstm.csv", index=False)
print("[INFO] saved /content/drive/MyDrive/week9_results_lstm.csv")
df_lstm


/tmp/ipython-input-116966612.py:9: DeprecationWarning: torch.ao.quantization is deprecated and will be removed in 2.10. 
For migrations of users: 
1. Eager mode quantization (torch.ao.quantization.quantize, torch.ao.quantization.quantize_dynamic), please migrate to use torchao eager mode quantize_ API instead 
2. FX graph mode quantization (torch.ao.quantization.quantize_fx.prepare_fx,torch.ao.quantization.quantize_fx.convert_fx, please migrate to use torchao pt2e quantization API instead (prepare_pt2e, convert_pt2e) 
3. pt2e quantization has been migrated to torchao (https://github.com/pytorch/ao/tree/main/torchao/quantization/pt2e) 
see https://github.com/pytorch/ao/issues/2259 for more details
  return tq.quantize_dynamic(model.cpu(), {nn.Linear}, dtype=torch.qint8)


[INFO] saved /content/drive/MyDrive/week9_results_lstm.csv


,Model,Variant,Latency_ms,Size_MB,Accuracy,F1
0,LSTM,Baseline,24.008036,0.227002,0.825,0.824010
1,LSTM,Quantized,39.300203,0.223505,0.825,0.824010
2,LSTM,Pruned(0.4),15.881777,0.236858,0.815,0.813315


In [ ]:
df_all = pd.concat([df_cnn, df_lstm], ignore_index=True)
out = "/content/drive/MyDrive/week9_results_all.csv"
df_all.to_csv(out, index=False)
print("[INFO] Saved dashboard CSV:", out)
df_all


[INFO] Saved dashboard CSV: /content/drive/MyDrive/week9_results_all.csv


,Model,Variant,Latency_ms,Size_MB,Accuracy,F1
0,CNN,Baseline,15.522480,0.042601,0.835,0.831937
1,CNN,Quantized,1.754761,0.037719,0.840,0.836801
2,CNN,Pruned(0.4),0.875950,0.051088,0.810,0.801234
3,LSTM,Baseline,24.008036,0.227002,0.825,0.824010
4,LSTM,Quantized,39.300203,0.223505,0.825,0.824010
5,LSTM,Pruned(0.4),15.881777,0.236858,0.815,0.813315
